# Steps to perform SAST using Sonarqube

## Check if Sonarqube is up

In [ ]:
!docker ps | grep sonarqube

## Start the container if not available

In [ ]:
!docker run -d --name sonarqube -p 9000:9000 sonarqube

### Run the below cell if you already have a container

In [ ]:
!docker start sonarqube

## Setting variables

In [86]:
import os

os.environ["SONARQUBE_URL"] = #### Enter sonarqube server url
os.environ["USERNAME"] = #### Enter your sonarqube username
os.environ["PASSWORD"] = #### Enter your sonarqube password
os.environ["YOUR_PROJECT_KEY"] = #### Enter your project key(can be the same as project name)
os.environ["YOUR_PROJECT_NAME"] = #### Enter your project name

## Creating user token

This token wil be used across multiple projects

In [ ]:
import os
!curl -u $USERNAME:$PASSWORD -X POST $SONARQUBE_URL/api/user_tokens/generate -d name=automation_token > /tmp/sonar.json
!jq -r '.token' /tmp/sonar.json > /tmp/token.txt
token = open('/tmp/token.txt', 'r').read()
os.environ["TOKEN"] = token
!rm /tmp/token.txt /tmp/sonar.json

### Get user token

Run the below cell to get the generated token. Copy it and keep it in hand

In [ ]:
import os
print(os.environ.get("TOKEN"))

### Run the below cell, if you already have a token

In [ ]:
import os

os.environ["TOKEN"] = ### Enter token

## Create project

The command below will create a new project using the main branch. 

Note- `If you require a different branch, please visit the sonarqube server page and create the project manually.`

In [ ]:
!curl -u "$USERNAME":"$PASSWORD" -X POST "$SONARQUBE_URL/api/projects/create?name=$YOUR_PROJECT_NAME&project=$YOUR_PROJECT_KEY"

## Run Scan
Move to your project directory to perform the scan

### For maven builds

In [ ]:
import os

artifact_dir = os.environ.get("BESMAN_ARTIFACT_DIR")
os.chdir(artifact_dir)
!mvn clean verify sonar:sonar -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.projectName="$YOUR_PROJECT_NAME" -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.login="$TOKEN"


### For gradle builds

Running an analysis with Gradle is straighforward. You just need to declare the `org.sonarqube` plugin in your `build.gradle` or `build.gradle.kts` file:

**build.gradle**
```json
plugins {
  id "org.sonarqube" version "5.0.0.4638"
}
```

**build.gradle.kts**

```json
plugins {
  id("org.sonarqube") version "5.0.0.4638"
}
```


In [ ]:
import os

artifact_dir = os.environ.get("BESMAN_ARTIFACT_DIR")
os.chdir(artifact_dir)
!./gradlew sonar -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.projectName='$YOUR_PROJECT_NAME' -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.token=$TOKEN

### For others(JS, TS, Go, Python, PHP)

In [ ]:
import os

artifact_dir = os.environ.get("BESMAN_ARTIFACT_DIR")
os.chdir(artifact_dir)
!sonar-scanner -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.sources=. -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.token="$TOKEN"

## View Report

Once the assessment is done, you can visit the Sonarqube server and see your project scan report

## Download Report

In [ ]:
import os
import requests
import json

# Get the SonarQube URL from the environment variable
sonarqube_url_env = os.environ.get('SONARQUBE_URL')

username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")
detailed_report = os.environ.get("DETAILED_REPORT_PATH")
project_name = os.environ.get("YOUR_PROJECT_NAME")
# Specify the initial page size
page_size = 500  # For example, set the initial page size to 500

url = f"{sonarqube_url_env}/api/issues/search?componentKeys={project_name}&ps={page_size}&p=1"
vuln_url = f"{sonarqube_url_env}/api/issues/search?componentKeys={project_name}&types=VULNERABILITY&ps={page_size}&p=1"


# Get first page data
try:
    response_first = requests.get(url, auth=(username, password))
    response_first.raise_for_status()  # Raise an exception for non-200 responses
    data_first = response_first.json()
except requests.RequestException as e:
    print("Error making the request:", e)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Raw response content:", response_first.content)

# Get first page data for type VULNERABILITY 
try:
    response_vuln_first = requests.get(vuln_url, auth=(username, password))
    response_vuln_first.raise_for_status()  # Raise an exception for non-200 responses
    data_vuln_first = response_vuln_first.json()
except requests.RequestException as e:
    print("Error making the request:", e)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Raw response content:", response_vuln_first.content)

# Initialize variables for tracking the current page and the aggregated issues
all_issues = []

# Fetch data from all pages
current_page = 2
while True:
    # Construct the API URL with the current page size and page number
    url = f"{sonarqube_url_env}/api/issues/search?componentKeys={project_name}&ps={page_size}&p={current_page}"

    # Make the API request
    try:
        response = requests.get(url, auth=(username, password))
        response.raise_for_status()  # Raise an exception for non-200 responses
        data = response.json()

        # Extract the "issues" field and add it to the aggregated issues
        if "issues" in data:
            data_first["issues"].extend(data["issues"])

        # Increment the page number for the next iteration
        current_page += 1

        # Check if there are more pages to fetch
        if len(data["issues"]) < page_size:
            break  # Exit the loop if the last page has been reached
    except requests.RequestException as e:
        print("Error making the request:", e)
        break  # Exit the loop on request exception
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        print("Raw response content:", response.content)
        break  # Exit the loop on JSON decoding error

# Get the vulnerability data from page 2 onwards 
current_page = 2
while True:
    # Construct the API URL with the current page size and page number
    vuln_url = f"{sonarqube_url_env}/api/issues/search?componentKeys={project_name}&types=VULNERABILITY&ps={page_size}&p={current_page}"

    # Make the API request
    try:
        vuln_response = requests.get(vuln_url, auth=(username, password))
        vuln_response.raise_for_status()  # Raise an exception for non-200 responses
        vuln_data = vuln_response.json()


        # Extract the "issues" field and add it to the aggregated issues
        if vuln_data["total"] ==  0:
            break
        if len(vuln_data["issues"]) > 0:
            data_vuln_first["issues"].extend(vuln_data["issues"])

        # Increment the page number for the next iteration
        current_page += 1

        # Check if there are more pages to fetch
        if len(vuln_data["issues"]) < page_size:
            break  # Exit the loop if the last page has been reached
    except requests.RequestException as e:
        print("Error making the request:", e)
        break  # Exit the loop on request exception
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        print("Raw response content:", vuln_response.content)
        break  # Exit the loop on JSON decoding error


data_first["issues"].extend(data_vuln_first["issues"])
with open(detailed_report, "w") as file:
    json.dump(data_first, file, indent=4)
